In [ ]:
import pandas as pd
from multicall.call import Call
from multicall.fetch_multicall_across_blocks import fetch_save_and_return
from multicall.constants import W3
from helpers import TEST_CACHE_PATH, TEST_BLOCK


def identity_function(x):
    return x


uniswap_v3_usdc_weth_pool = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"
weth = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
usdc = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"

weth_bal = Call(
    weth,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "weth_bal",
    identity_function,
)

block_to_check = TEST_BLOCK

# Call In Jupyter

In [ ]:
from multicall.utils import time_function


@time_function
def a():
    print(weth_bal(W3, block_to_check, cache=TEST_CACHE_PATH))

In [ ]:
# first
a()

In [ ]:
# second
a()

# Test Mulitcall In jupyter

In [ ]:
usdc_bal = Call(
    usdc,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "usdc_bal",
    identity_function,
)

from multicall.multicall import Multicall


m = Multicall([weth_bal, usdc_bal])


@time_function
def b():
    print(m(W3, block_to_check, cache=TEST_CACHE_PATH))

In [ ]:
# first
b()

In [ ]:
# second
b()

# Test Fetch and Save


In [ ]:
# first
fetch_save_and_return(
    [usdc_bal, weth_bal], [block_to_check - 1, block_to_check, block_to_check + 1], W3, cache=TEST_CACHE_PATH
)

In [ ]:
# second
fetch_save_and_return(
    [usdc_bal, weth_bal], [block_to_check - 1, block_to_check, block_to_check + 1], W3, cache=TEST_CACHE_PATH
)

In [ ]:
def norm_18(x):
    return x / 1e18


def norm_6(x):
    return x / 6


weth_bal_different_handle_function = Call(
    weth,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "weth_bal",
    norm_18,
)


usdc_bal_different_handle_function = Call(
    usdc,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "usdc_bal",
    norm_6,
)

In [ ]:
# third
fetch_save_and_return(
    [usdc_bal_different_handle_function, weth_bal],
    [block_to_check - 1, block_to_check, block_to_check + 1],
    W3,
    cache=TEST_CACHE_PATH,
)

# Test with size,

- Get the USDC, and WETH balance for 1_000 addresses
- also get the get the addr.balanceOf(addr) for all the addresses

- 1k blocks and 1k function calls, 1M points of state

In [ ]:
1000 * 1000

In [ ]:
df = pd.read_csv("../many_weth_transfers_more.csv")
all_tos = df["from"].unique()[:1_000]
len(all_tos)

In [ ]:
calls = []
for person in all_tos:
    calls.append(Call(usdc, "balanceOf(address)(uint256)", (person), f"{person}_USDC", norm_6))
    calls.append(Call(weth, "balanceOf(address)(uint256)", (person), f"{person}_WETH", norm_18))

blocks = [18_000_000 + i * 100 for i in range(10)]
len(calls), len(blocks)  # wide and narrow only 20k calls, 10 blocks

In [ ]:
# first
fetch_save_and_return(calls, blocks, W3, cache=TEST_CACHE_PATH)

In [ ]:
# second
fetch_save_and_return(calls, blocks, W3, cache=TEST_CACHE_PATH)